## 1. Web Access

### 1. Modules & Variables

In [ ]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크
# https://github.com/ultrafunkamsterdam/undetected-chromedriver


In [1]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC  # noqa

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [15]:
info = open('id_pw.txt').readlines()

account_1 = info[0].split(' ')[1].rstrip('\n')
password_1 = info[1].split(' ')[1].rstrip('\n')

account_2 = info[2].split(' ')[1].rstrip('\n')
password_2 = info[3].split(' ')[1].rstrip('\n')

In [16]:
password_1

'flapaqj1!%'

### 2. Web Access

In [17]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [18]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
btn_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
btn_sign_in.click()

# 계정 이메일 입력
input_account = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
input_account.send_keys(account_1)

# [다음] 버튼 클릭해 패스워드 입력
btn_next_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
btn_next_password.click()

# 계정 비밀번호 입력
input_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
input_password.send_keys(password_1)

# [다음] 버튼 입력해 로그인 클릭
btn_next_sign_in = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
btn_next_sign_in.click()


In [11]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지
try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


In [19]:
driver.refresh()


### 3. Data Collection

#### 1) Variables

In [50]:
# Variables (1) - record metadata

# 시청일자 - 목록 content에서 [data-date] 값 추출
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>

id_str_pattern = "https://www.youtube.com/watch?v="

In [39]:
# Variables (2) Category modules and vars

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

key = open('api_key.txt').readline()

DEVELOPER_KEY = key
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=DEVELOPER_KEY)

#### 2) Functions

In [40]:
# Function (1) - date & day

def get_day(weekday=datetime.today().weekday()):
    list = ['월', '화', '수', '목', '금', '토', '일']
    return list[weekday]

def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d').strftime('%Y-%m-%d')


In [52]:
# Function (2) - Category Id & Title

def get_catId(input_id):
    video = youtube.videos().list(
        part = 'snippet',
        id = input_id
        ).execute()

    return video['items'][0]['snippet']['categoryId']

def get_catTitle(num):
    catId = youtube.videoCategories().list(
    id = num,
    part = 'snippet'
    ).execute()
    
    try:
        return catId['items'][0]['snippet']['title']
    except:
        return ''

In [ ]:
# Function (3) - Watched_YN classifier

#### 2) DataFrame

In [21]:
import pandas as pd
columns_rec = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category_id', 'category_title'] # 'watched_YN'
df_record = pd.DataFrame(columns=columns_rec)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title


In [22]:
columns_ser = ['searchword', 'search_date', 'search_time']
df_search = pd.DataFrame(columns=columns_ser)
df_search.head()


,searchword,search_date,search_time


#### 3) Web Data Collecting

In [42]:
# # 목표 갯수만큼 시청 목록(레코드) 호출

# num_target_data = 1000  # int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))
# num_press_end = int(num_target_data / 100)

# for n in range(num_press_end - 1):
#     driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
#     time.sleep(2) # 전체 페이지가 뜨도록 Wait

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')


호출된 레코드 수: 100


In [51]:
id = contents[1]
id

'SK1DmgQcYF4'

In [53]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime): # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            channel = content.find('div', class_=nm_class_channel).text 
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            # watched_YN = input(f'{title}을 끝까지 시청하셨나요? \t')

            id = content.find('a')['href'].replace(id_str_pattern, '')
            category_id = get_catId(id) #input(f'{title}의 Category는? \t') # 영상 분류 Model 활용을 위해
            category_title = get_catTitle(category_id)

            df_record.loc[idx_rec] = [title, channel,
                                      running_time, watch_date, watch_time, category_id, category_title] #watched_YN
            idx_rec += 1

        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Search Record
1: Watch Record
2: Watch Record
3: Watch Record
4: Watch Record
5: Ad
6: Ad
7: Watch Record
8: Watch Record
9: Ad
10: Watch Record
11: Watch Record
12: Watch Record
13: Watch Record
14: Watch Record
15: Watch Record
16: Watch Record
17: Watch Record
18: Watch Record
19: Watch Record
20: Watch Record
21: Watch Record
22: Watch Record
23: Watch Record
24: Watch Record
25: Watch Record
26: Watch Record
27: Watch Record
28: Watch Record
29: Watch Record
30: Watch Record
31: Watch Record
32: Watch Record
33: Watch Record
34: Watch Record
35: Watch Record
36: Watch Record
37: Watch Record
38: Watch Record
39: Watch Record
40: Watch Record
41: Watch Record
42: Watch Record
43: Watch Record
44: Watch Record
45: Watch Record
46: Watch Record
47: Watch Record
48: Watch Record
49: Community Post 	 Watch time: 오후 1:50
50: Watch Record
51: Watch Record
52: Watch Record
53: Watch Record
54: Watch Record
55: Watch Record
56: Watch Record
57: Watch Record
58: Watch Record
59: Watch Re

In [55]:
# df_record.to_csv('230212_watch_record.csv', encoding='utf-8')
df_record.head(50)


,title,channel,running_time,watch_date,watch_time,category_id,category_title
0,데이터 엔지니어는 무슨 일을 하는 사람일까? | 데이터엔지니어 직무 소개(feat....,메이크센스 MakeSense,5:39,2023-02-12,오전 12:49,26,Howto & Style
1,수상할정도로 벨코즈를 잘하는 곰,박나나,11:01,2023-02-11,오후 10:39,20,Gaming
2,마스터이 명상 무한으로 사용하는 좀비 마스터이 ㅋㅋㅋㅋ [테스터훈 뉴메타],테스터훈 마이너스,10:55,2023-02-11,오후 10:27,20,Gaming
3,"출장 수리 사기, 결국 고소했습니다",뻘짓연구소,11:13,2023-02-11,오후 9:18,28,Science & Technology
4,응애 | 젠지 vs. KT 게임1 하이라이트 | 02.11 | 2023 LCK 스프...,LCK,6:34,2023-02-11,오후 9:12,20,Gaming
5,"""𝟙𝟘𝟘"" | 젠지 vs. KT 게임3 하이라이트 | 02.11 | 2023 LCK...",LCK,10:55,2023-02-11,오후 9:01,20,Gaming
6,일 왜함? - 러스트(2),랄로,43:46,2023-02-11,오후 9:00,20,Gaming
7,요네는 돌아오는거야,PAKA,24:43,2023-02-11,오후 8:35,20,Gaming
8,럭스1위 여왕럭스 본캐 그마 솔로랭크,여왕럭스,4:53:10,2023-02-11,오후 8:14,20,Gaming
9,이 바텀 조합 살벌한데..? 페이커의 딩거 서폿! (feat.구마유시) [Faker...,T1 Faker,6:47,2023-02-11,오후 8:07,20,Gaming


In [ ]:
df_search.to_csv('230211_search_record.csv', encoding='utf-8')
df_search


In [26]:
print(
    f"입력한 시청기록 레코드는 {num_target_data}개, \n실제 시청한 영상은 {len(df_record)}개, \n광고는 {num_ad}개입니다.")


입력한 시청기록 레코드는 1000개, 
실제 시청한 영상은 418개, 
광고는 404개입니다.


In [ ]:
# ERROR - 커뮤니티 글 확인 기록
# 채널은 있는데, 영상이 없음
"""
80: NO ads
지금 이 순간 대생성AI 버블 시대가 열렸습니다. 닷컴버블, 스마트폰 혁명, 블록체인 열풍을 놓치신 개발자분들 이번 기회는 꼭 잡으시길 바랍니다!
🎬영상보기: https://youtu.be/4xU0mGr4g...
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[33], line 9
      7     print(f'{idx}: NO ads')
      8     print(content.find('a', class_=nm_class_title).text)
----> 9     print(content.find('div', class_=nm_class_RunningTime).text)
     10     print(' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2]))
     12 else:

AttributeError: 'NoneType' object has no attribute 'text'
"""


### Spare codes

In [ ]:
btn_acccount_info = driver.find_element(
    By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/div[2]/div/a')

btn_sign_out = ''
# html document, iframe 태그 안에 들어있어 확인이 어려움.
# 여러 계정을 같이 분석할 경우 driver 초기화 방식으로 진행하자.


In [72]:
driver.close()


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all('c-wiz', class_='xDtZAf')
for idx, content in enumerate(contents):
    if content.find('div', 'SiEggd'):
        if content.find('div', class_=nm_class_RunningTime):
            print(f"{idx}: Watch Record")
            print(f"Watch Date: {get_date(content['data-date'])}")
            print(f"Title: {content.find('a', class_=nm_class_title).text}")
            print(
                f"Running Time: {content.find('div', class_=nm_class_RunningTime).text}")
            print(
                f"Watch Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Community Post")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search")
            print(f"Search Date: {get_date(content['data-date'])}")
            print(
                f"Search Word: {content.find('a', class_=nm_class_title).text}")
            print(
                f"Search Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Ad")

    print(' ')


0: Ad
 
1: Ad
 
2: Watch Record
Watch Date: 2023-02-04
Title: 초가스 키우기 | 디알엑스 vs. 한화생명 게임1 하이라이트 | 02.04 | 2023 LCK 스프링 스플릿
Running Time: 12:33
Watch Time: 오후 8:19
 
3: Ad
 
4: Ad
 
5: Watch Record
Watch Date: 2023-02-04
Title: 신궁 | 농심 vs. T1 게임1 하이라이트 | 02.04 | 2023 LCK 스프링 스플릿
Running Time: 9:38
Watch Time: 오후 8:10
 
6: Ad
 
7: Watch Record
Watch Date: 2023-02-04
Title: [스티브 잡스] A급 인재와 B급, C급 직원의 차이 (한영 자막)
Running Time: 7:27
Watch Time: 오후 8:02
 
8: Search
Search Date: 2023-02-04
Search Word: 파카
Search Time: 오전 2:02
 
9: Watch Record
Watch Date: 2023-02-04
Title: Crimson Raptor Champion Spotlight | Parody - League of Legends
Running Time: 9:45
Watch Time: 오전 1:58
 
10: Ad
 
11: Watch Record
Watch Date: 2023-02-04
Title: 젠지 vs. 리브 샌박 | 매치26 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 45:49
Watch Time: 오전 12:54
 
12: Watch Record
Watch Date: 2023-02-04
Title: KT vs. 광동 | 매치25 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 26:07
Watch Time: 오전 12:53
 
13: Ad
 
14: Ad
 
15: Watch Reco

### Troubleshooting

아이디 / 비밀번호 입력 없이 로그인을 직접 한다면?
- 보안 문제
- 로그인 한 후 driver 초기화 문제
    - 1) driver 재시작
    - 2) 